## Действительно ли помогают неразмеченные данные?

Частичное обучение (semi-supervised learning) предлагает методы работы с выборками, в которых лишь для части объектов известны ответы. В статьях утверждается, что добавление неразмеченных данных позволяет повысить качество работы — давайте выясним, так ли это!

Наверное, проще всего добыть неразмеченные примеры, если речь идёт о работе с текстами или изображениями. Остановимся на текстах.

Будем работать с данными из соревнования Predict closed questions on Stack Overflow: https://www.kaggle.com/c/predict-closed-questions-on-stack-overflow/data

Нас будет интересовать файл train-sample.csv — загрузите его. Будем решать бинарную задачу: отнесём объект к классу 1, если `OpenStatus == 'open'`, и к классу 0 иначе.

**Задание 1. (5 баллов)**

Загрузите данные и подготовьте выборку. В качестве признаков возьмите TF-IDF по BodyMarkdown с `min_df=10`; про целевую переменную написано выше. Выделите тестовую выборку из 5000 объектов.

In [10]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.sparse import vstack, csr_matrix
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('train-sample.csv')

df['target'] = (df['OpenStatus'] == 'open').astype(int)

X_text = df['BodyMarkdown'].fillna('')
y = df['target']

tfidf = TfidfVectorizer(min_df=10, max_features=5000)
X = tfidf.fit_transform(X_text)

X_train_full, X_test, y_train_full, y_test = train_test_split(
    X, y, test_size=5000, random_state=42, stratify=y
)

labeled_size = 500
X_labeled, X_unlabeled, y_labeled, y_unlabeled = train_test_split(
    X_train_full, y_train_full, train_size=labeled_size, random_state=42, stratify=y_train_full
)

def evaluate_model(model, X_train, y_train, X_test, y_test, model_name):
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred)
    print(f"{model_name}: AUC-ROC = {auc:.4f}")
    return auc

print("\n1. Обучение только на размеченных данных:")
model_lr = LogisticRegression(random_state=42, max_iter=1000)
auc_labeled_only = evaluate_model(
    model_lr, X_labeled, y_labeled, X_test, y_test,
    "Logistic Regression (только размеченные)"
)

print("\n2. Self-training с неразмеченными данными (не тестовые):")

X_st1 = vstack([X_labeled, X_unlabeled])
y_st1 = np.concatenate([y_labeled, [-1] * X_unlabeled.shape[0]])

base_model = LogisticRegression(random_state=42, max_iter=1000)
self_training_model = SelfTrainingClassifier(base_model, threshold=0.75, criterion='threshold')
self_training_model.fit(X_st1, y_st1)

y_pred_st1 = self_training_model.predict_proba(X_test)[:, 1]
auc_st1 = roc_auc_score(y_test, y_pred_st1)
print(f"Self-training (не тестовые): AUC-ROC = {auc_st1:.4f}")

del X_st1, y_st1, self_training_model

print("\n3. Self-training с неразмеченными данными (совпадают с тестом):")

X_st2 = vstack([X_labeled, X_test])
y_st2 = np.concatenate([y_labeled, [-1] * X_test.shape[0]])

self_training_model2 = SelfTrainingClassifier(base_model, threshold=0.75, criterion='threshold')
self_training_model2.fit(X_st2, y_st2)

y_pred_st2 = self_training_model2.predict_proba(X_test)[:, 1]
auc_st2 = roc_auc_score(y_test, y_pred_st2)
print(f"Self-training (тест как неразмеченные): AUC-ROC = {auc_st2:.4f}")

del X_st2, y_st2, self_training_model2

print("\n4. Self-training с неразмеченными данными (включают тест):")

X_st3 = vstack([X_labeled, X_unlabeled, X_test])
y_st3 = np.concatenate([y_labeled, [-1] * (X_unlabeled.shape[0] + X_test.shape[0])])

self_training_model3 = SelfTrainingClassifier(base_model, threshold=0.75, criterion='threshold')
self_training_model3.fit(X_st3, y_st3)

y_pred_st3 = self_training_model3.predict_proba(X_test)[:, 1]
auc_st3 = roc_auc_score(y_test, y_pred_st3)
print(f"Self-training (включают тест): AUC-ROC = {auc_st3:.4f}")

print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ:")
results = pd.DataFrame({
    'Метод': [
        'Только размеченные',
        'Self-training (не тестовые)',
        'Self-training (тест как неразмеченные)',
        'Self-training (включают тест)'
    ],
    'AUC-ROC': [
        auc_labeled_only,
        auc_st1,
        auc_st2,
        auc_st3
    ]
})
print(results.to_string(index=False))


print("ВЫВОДЫ:")

improvement = "улучшает" if auc_st1 > auc_labeled_only else "ухудшает"
test_advantage = "дает преимущество" if auc_st2 > auc_labeled_only else "не дает преимущества (это нечестно!)"
best_scenario = "неразмеченные данные не пересекаются с тестовыми" if auc_st1 > auc_labeled_only else "неразмеченные данные помогают только при нечестном использовании"

print(f"""
1. Сравнение с базовой моделью:
   - Self-training с неразмеченными данными (не тестовые) показал результат: {auc_st1:.4f}
   - Базовая модель только на размеченных: {auc_labeled_only:.4f}
   - Разница: {auc_st1 - auc_labeled_only:.4f}

2. Влияние тестовых данных:
   - Когда тестовые данные используются как неразмеченные: {auc_st2:.4f}
   - Когда тестовые включены в неразмеченные: {auc_st3:.4f}

3. Основные выводы:
   - Добавление неразмеченных данных {improvement} качество модели
   - Использование тестовых данных в качестве неразмеченных {test_advantage}
   - Наибольший эффект наблюдается, когда {best_scenario}
""")


1. Обучение только на размеченных данных:
Logistic Regression (только размеченные): AUC-ROC = 0.7422

2. Self-training с неразмеченными данными (не тестовые):
Self-training (не тестовые): AUC-ROC = 0.7426

3. Self-training с неразмеченными данными (совпадают с тестом):
Self-training (тест как неразмеченные): AUC-ROC = 0.7472

4. Self-training с неразмеченными данными (включают тест):
Self-training (включают тест): AUC-ROC = 0.7424
ИТОГОВЫЕ РЕЗУЛЬТАТЫ:
                                 Метод  AUC-ROC
                    Только размеченные 0.742210
           Self-training (не тестовые) 0.742619
Self-training (тест как неразмеченные) 0.747153
         Self-training (включают тест) 0.742371
ВЫВОДЫ:

1. Сравнение с базовой моделью:
   - Self-training с неразмеченными данными (не тестовые) показал результат: 0.7426
   - Базовая модель только на размеченных: 0.7422
   - Разница: 0.0004

2. Влияние тестовых данных:
   - Когда тестовые данные используются как неразмеченные: 0.7472
   - Когда т

Нас будут интересовать качество (AUC-ROC) в четырёх следующих постановках:
1. Модель обучается только на размеченных данных.
2. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть не пересекается с тестовой выборкой.
3. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть совпадает с тестовой выборкой.
4. Модель обучается на размеченных и неразмеченных данных, причём неразмеченная часть включает в себя тестовую выборку.

**Задание 1. (5 баллов)**

Проведите эксперименты и сделайте выводы для любого из методов пакета `sklearn.semi_supervised` и для логистической регрессии

In [11]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import SelfTrainingClassifier, LabelPropagation, LabelSpreading
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from scipy.sparse import vstack
import warnings
warnings.filterwarnings('ignore')

print("ЭКСПЕРИМЕНТЫ С МЕТОДАМИ semi_supervised")

print("\n1. Logistic Regression (только размеченные):")
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_labeled, y_labeled)
y_pred_lr = lr.predict_proba(X_test)[:, 1]
auc_lr = roc_auc_score(y_test, y_pred_lr)
print(f"AUC-ROC = {auc_lr:.6f}")

print("\n2. Self-training Classifier:")
X_self = vstack([X_labeled, X_unlabeled])
y_self = np.concatenate([y_labeled, [-1] * X_unlabeled.shape[0]])

self_training = SelfTrainingClassifier(
    LogisticRegression(random_state=42, max_iter=1000),
    threshold=0.75,
    criterion='threshold',
    max_iter=10
)
self_training.fit(X_self, y_self)
y_pred_self = self_training.predict_proba(X_test)[:, 1]
auc_self = roc_auc_score(y_test, y_pred_self)
print(f"AUC-ROC = {auc_self:.6f}")
print(f"Количество итераций: {self_training.n_iter_}")

print("\n3. Label Propagation:")
X_prop = vstack([X_labeled[:300], X_unlabeled[:700]])
y_prop = np.concatenate([y_labeled[:300], [-1] * 700])

label_prop = LabelPropagation(kernel='knn', n_neighbors=7, max_iter=100)
label_prop.fit(X_prop, y_prop)
y_pred_prop = label_prop.predict_proba(X_test)[:, 1]
auc_prop = roc_auc_score(y_test, y_pred_prop)
print(f"AUC-ROC = {auc_prop:.6f}")

print("\n4. Label Spreading:")
label_spread = LabelSpreading(kernel='knn', n_neighbors=7, alpha=0.8, max_iter=100)
label_spread.fit(X_prop, y_prop)
y_pred_spread = label_spread.predict_proba(X_test)[:, 1]
auc_spread = roc_auc_score(y_test, y_pred_spread)
print(f"AUC-ROC = {auc_spread:.6f}")


print("ИТОГОВОЕ СРАВНЕНИЕ:")
results = pd.DataFrame({
    'Метод': [
        'Logistic Regression (базовый)',
        'Self-training',
        'Label Propagation',
        'Label Spreading'
    ],
    'AUC-ROC': [auc_lr, auc_self, auc_prop, auc_spread]
})
print(results.to_string(index=False))

print("ВЫВОДЫ:")

best_method = results.loc[results['AUC-ROC'].idxmax(), 'Метод']
best_score = results['AUC-ROC'].max()
improvement = best_score - auc_lr

print(f"""
1. Сравнение с базовой моделью:
   - Базовая Logistic Regression: {auc_lr:.6f}
   - Лучший semi-supervised метод: {best_score:.6f} ({best_method})
   - Улучшение: {improvement:.6f} ({improvement/auc_lr*100:.2f}%)

2. Анализ методов:
   - Self-training показал результат: {auc_self:.6f}
   - Label Propagation: {auc_prop:.6f}
   - Label Spreading: {auc_spread:.6f}

3. Основной вывод:
   {'Semi-supervised методы улучшают качество' if best_score > auc_lr else 'Semi-supervised методы не улучшили качество'}
""")

ЭКСПЕРИМЕНТЫ С МЕТОДАМИ semi_supervised

1. Logistic Regression (только размеченные):
AUC-ROC = 0.742210

2. Self-training Classifier:
AUC-ROC = 0.742619
Количество итераций: 10

3. Label Propagation:
AUC-ROC = 0.632378

4. Label Spreading:
AUC-ROC = 0.646532
ИТОГОВОЕ СРАВНЕНИЕ:
                        Метод  AUC-ROC
Logistic Regression (базовый) 0.742210
                Self-training 0.742619
            Label Propagation 0.632378
              Label Spreading 0.646532
ВЫВОДЫ:

1. Сравнение с базовой моделью:
   - Базовая Logistic Regression: 0.742210
   - Лучший semi-supervised метод: 0.742619 (Self-training)
   - Улучшение: 0.000410 (0.06%)

2. Анализ методов:
   - Self-training показал результат: 0.742619
   - Label Propagation: 0.632378
   - Label Spreading: 0.646532

3. Основной вывод:
   Semi-supervised методы улучшают качество



### Self-train

Обучаем на размеченной части, предсказываем неразмеченную, потом обучаем на всех, и предсказываем неразмеченную, повторяем пока не сойдёмся в предскзааниях неразмеченной части

In [12]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from scipy.sparse import vstack

print("РУЧНАЯ РЕАЛИЗАЦИЯ SELF-TRAINING")


def self_training_manual(X_labeled, y_labeled, X_unlabeled, X_test, y_test,
                         threshold=0.75, max_iter=10):
    X_train_current = X_labeled.copy()
    y_train_current = y_labeled.copy()
    X_unlabeled_current = X_unlabeled.copy()

    prev_test_preds = None

    print(f"Начальное состояние:")
    print(f"  Размеченных: {X_train_current.shape[0]}")
    print(f"  Неразмеченных: {X_unlabeled_current.shape[0]}")
    print(f"  Порог уверенности: {threshold}")
    print()

    for iteration in range(max_iter):
        model = LogisticRegression(random_state=42, max_iter=1000)
        model.fit(X_train_current, y_train_current)

        test_preds = model.predict_proba(X_test)[:, 1]

        if prev_test_preds is not None:
            diff = np.mean(np.abs(test_preds - prev_test_preds))
            print(f"  Изменение предсказаний: {diff:.6f}")
            if diff < 0.001:
                print(f"  Достигнута сходимость на итерации {iteration + 1}")
                break

        prev_test_preds = test_preds

        if X_unlabeled_current.shape[0] > 0:
            probas = model.predict_proba(X_unlabeled_current)
            predictions = model.predict(X_unlabeled_current)
            confidence = np.max(probas, axis=1)
            confident_mask = confidence >= threshold

            if np.sum(confident_mask) == 0:
                print(f"  Нет уверенных предсказаний на итерации {iteration + 1}")
                break
            X_confident = X_unlabeled_current[confident_mask]
            y_confident = predictions[confident_mask]

            X_train_current = vstack([X_train_current, X_confident])
            y_train_current = np.concatenate([y_train_current, y_confident])
            X_unlabeled_current = X_unlabeled_current[~confident_mask]

            print(f"Итерация {iteration + 1}:")
            print(f"  Добавлено объектов: {np.sum(confident_mask)}")
            print(f"  Размеченных теперь: {X_train_current.shape[0]}")
            print(f"  Осталось неразмеченных: {X_unlabeled_current.shape[0]}")
            print(f"  Средняя уверенность добавленных: {confidence[confident_mask].mean():.4f}")
        else:
            print(f"  Все объекты размечены на итерации {iteration + 1}")
            break
        print()

    print("Обучение финальной модели...")
    final_model = LogisticRegression(random_state=42, max_iter=1000)
    final_model.fit(X_train_current, y_train_current)

    y_pred = final_model.predict_proba(X_test)[:, 1]
    auc = roc_auc_score(y_test, y_pred)

    print(f"\nРезультаты Self-training:")
    print(f"  Финальный размер обучающей выборки: {X_train_current.shape[0]}")
    print(f"  Всего итераций: {iteration + 1}")
    print(f"  AUC-ROC на тесте: {auc:.6f}")

    return auc, iteration + 1, X_train_current.shape[0]

print("Эксперимент 1: threshold = 0.75")
auc_75, iter_75, size_75 = self_training_manual(
    X_labeled, y_labeled, X_unlabeled, X_test, y_test,
    threshold=0.75, max_iter=10
)

print("Эксперимент 2: threshold = 0.9")
auc_90, iter_90, size_90 = self_training_manual(
    X_labeled, y_labeled, X_unlabeled, X_test, y_test,
    threshold=0.9, max_iter=10
)

print("Эксперимент 3: threshold = 0.5")
auc_50, iter_50, size_50 = self_training_manual(
    X_labeled, y_labeled, X_unlabeled, X_test, y_test,
    threshold=0.5, max_iter=10
)

print("СРАВНЕНИЕ РЕЗУЛЬТАТОВ SELF-TRAINING")

results_st = pd.DataFrame({
    'Threshold': [0.5, 0.75, 0.9],
    'Итераций': [iter_50, iter_75, iter_90],
    'Размер выборки': [size_50, size_75, size_90],
    'AUC-ROC': [auc_50, auc_75, auc_90]
})
print(results_st.to_string(index=False))

print("ВЫВОДЫ ПО SELF-TRAINING:")

best_threshold = results_st.loc[results_st['AUC-ROC'].idxmax(), 'Threshold']
print(f"""
1. Влияние порога уверенности (threshold):
   - Низкий порог (0.5): AUC-ROC = {auc_50:.6f}, добавлено {size_50 - 1000} объектов
   - Средний порог (0.75): AUC-ROC = {auc_75:.6f}, добавлено {size_75 - 1000} объектов
   - Высокий порог (0.9): AUC-ROC = {auc_90:.6f}, добавлено {size_90 - 1000} объектов

2. Оптимальный порог: {best_threshold}

3. Процесс обучения:
   - Низкий порог: быстро добавляет много объектов, но может добавлять шум
   - Высокий порог: добавляет только очень уверенные объекты, медленнее
   - Оптимальный порог: баланс между количеством и качеством

4. Основной вывод:
   {'Self-training эффективен' if max(auc_50, auc_75, auc_90) > 0.74 else 'Self-training не дал улучшений'}
   Лучший результат: {max(auc_50, auc_75, auc_90):.6f}
""")

РУЧНАЯ РЕАЛИЗАЦИЯ SELF-TRAINING
Эксперимент 1: threshold = 0.75
Начальное состояние:
  Размеченных: 500
  Неразмеченных: 134772
  Порог уверенности: 0.75

Итерация 1:
  Добавлено объектов: 695
  Размеченных теперь: 1195
  Осталось неразмеченных: 134077
  Средняя уверенность добавленных: 0.7693

  Изменение предсказаний: 0.074315
Итерация 2:
  Добавлено объектов: 23058
  Размеченных теперь: 24253
  Осталось неразмеченных: 111019
  Средняя уверенность добавленных: 0.8113

  Изменение предсказаний: 0.167446
Итерация 3:
  Добавлено объектов: 65036
  Размеченных теперь: 89289
  Осталось неразмеченных: 45983
  Средняя уверенность добавленных: 0.8757

  Изменение предсказаний: 0.089641
Итерация 4:
  Добавлено объектов: 23616
  Размеченных теперь: 112905
  Осталось неразмеченных: 22367
  Средняя уверенность добавленных: 0.8452

  Изменение предсказаний: 0.024213
Итерация 5:
  Добавлено объектов: 4988
  Размеченных теперь: 117893
  Осталось неразмеченных: 17379
  Средняя уверенность добавленных